# Load in the Data, Make Pandas-able
The raw data for this project is an ASE Database that holds the results of the water cluster calculations taken from a recent paper by [Rakshit et al.](https://doi.org/10.1063/1.5128378).
We need to convert these ASE `Atoms` objects, which list the atomic coordinates and energy, into a form with the graph structure.
This notebook contains the code to compute the graph structure for each entry in the database and save it all in an easily-accessible Pandas dataframe.

In [1]:
from graphsage.importing import create_graph, make_entry, make_tfrecord, make_nfp_network
from functools import partial
from multiprocessing import Pool
from ase.io.xyz import write_xyz
from ase.db import connect
from random import Random
from tqdm import tqdm
from io import StringIO
import tensorflow as tf
import pandas as pd
import numpy as np
import zipfile
import json
import gzip
import os

Configuration settings

In [2]:
total_graphs = 10 ** 3  # Number of graphs to read out from database for DF sample
val_fraction = 0.05  # Fraction of entries set aside for early stopping
test_fraction = 0.05  # Fraction of entries set aside for model testing
n_jobs = min(8, os.cpu_count())  # Number of processes to use

Create a random number generator

In [3]:
rng = np.random.RandomState(128)

## Get the ASE database from the ZIP file
We are going to uncompress it temporarily

In [4]:
data_zip = zipfile.ZipFile(os.path.join('data', 'input', 'ALL_geoms.zip'))

In [5]:
%%time
data_path = os.path.join('temp.db', 'water_db', 'ALL_geoms_all_sorted.db')
if not os.path.isfile(data_path):
    path_check = data_zip.extract('water_db/ALL_geoms_all_sorted.db', 'temp.db')
    assert path_check == data_path
    print(f'Extracted data to {data_path}')

CPU times: user 46 µs, sys: 25 µs, total: 71 µs
Wall time: 44.8 µs


In [6]:
%%time
ase_db = connect(data_path)
print(f'Connected to database with {len(ase_db)} records')

Connected to database with 4464740 records
CPU times: user 23.2 ms, sys: 34.9 ms, total: 58.1 ms
Wall time: 58.7 ms


## Convert ASE Objects to Networkx Graphs
The code here is adapated from the [Exalearn:Design Github page](https://github.com/exalearn/design/blob/16cfe21d85528c6004514d2985428566453b24a1/graph_descriptors/graph_builder.py).

Loop over the whole database

In [7]:
def pull_from_database(ase_db, rng, chunk_size=128, total=None):
    """Iterate over a large database
    
    Queries only a small chunk size at a time to prevent loading the 
    whole database into memory. 
    
    Args:
        ase_db (Connection): Connection to an ASE database
        rng (np.Random): Random number generator used to shuffle data
        chunk_size (int): Number of entries to retrieve per query
        total (int): Total number of entries to retrieve
    """
    # Figure out how many iterations we need to make
    if total is None:
        total = ase_db.count()
    
    # Generate the dataset
    starts = np.arange(0, total, chunk_size, dtype=np.int32)
    
    # Randomize the starts to help the diversity
    if rng is not None:
        rng.shuffle(starts)
    
    # Iterate through the whole database
    for start in starts:
        for a in ase_db.select(
            selection=[('id','>=', str(start)), ('id', '<', str(start+chunk_size))]):
            yield a.toatoms()

In [8]:
with Pool(n_jobs - 1) as p:  # Keep one thread reading the database
    graphs = list(tqdm(
        p.imap(make_entry, pull_from_database(ase_db, rng, total=total_graphs), chunksize=64),
        total=total_graphs
    ))

1023it [00:01, 642.13it/s]                                                                                                                                                                               


In [9]:
graphs = pd.DataFrame(graphs)

In [10]:
graphs.head(2)

,graph,energy,n_waters
0,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",-40.522038,6
1,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",-40.499618,6


### Save the DataFrame
Save the dataframe to disk

In [11]:
graphs.to_pickle(os.path.join('data', 'output', 'water_clusters.pkl.gz'))

## Save the Data as NFP-ready TensorFlow record objects
Save the whole file as a JSON-LD where each entry has the form:

```json
{
  'entry': 'entry number as an integer',
  'energy': 'energy as a float',
  'n_waters': 'number of water molecules as an integer', 
  'n_atom': 'number of atoms as an integer', 
  'n_bonds': 'number of bonds as an integer',
  'atom': 'List of atom types (0 -> Oxygen, 1 -> Hydrogen)',
  'bond': 'List of bond types (0 -> covalent, 1 -> Hydrogen)',
  'connectivity': 'List of connections between atoms, as a list of pairs of ints. Sorted ascending by column 0, then 1'
  'xyz': 'XYZ format version of the whole file.'
}
```

Also save a "coarsened" version of the network with a single node per water. 
Bonds are directional in the coarsened version, with a "donor" and "acceptor" bond type.
The format is otherwise identical but lacks the 'xyz' field

Pull a single cluster and make its network graph

In [12]:
atoms = next(pull_from_database(ase_db, None, total=2))

In [13]:
make_nfp_network(atoms)

{'n_water': 3,
 'n_atom': 9,
 'n_bond': 18,
 'atom': [0, 1, 1, 0, 1, 1, 0, 1, 1],
 'bond': [0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1],
 'connectivity': [[0, 1],
  [0, 2],
  [0, 8],
  [1, 0],
  [1, 3],
  [2, 0],
  [3, 1],
  [3, 4],
  [3, 5],
  [4, 3],
  [4, 6],
  [5, 3],
  [6, 4],
  [6, 7],
  [6, 8],
  [7, 6],
  [8, 0],
  [8, 6]],
 'energy': -15.9416428}

We can optionally coarsen that graph

In [14]:
make_nfp_network(atoms, coarsen=True)

{'n_water': 3,
 'n_atom': 3,
 'n_bond': 6,
 'atom': [0, 0, 0],
 'bond': [0, 1, 1, 0, 1, 0],
 'connectivity': [[0, 1], [0, 2], [1, 0], [1, 2], [2, 0], [2, 1]],
 'energy': -15.9416428}

Or just express the geometry

In [15]:
def geometry_record(atoms):
    """Create a JSON-ready record with the geometry and atomic types
    
    Args:
        atoms: ASE atoms object
    Returns:
        dictionary that can be easily serialized to JSON
    """
    z = atoms.get_atomic_numbers()
    return {
        'z': z.tolist(),
        'n_water': len(z) // 3,
        'n_atom': len(z),
        'atom': list(map([8, 1].index, atoms.get_atomic_numbers())),
        'coords': atoms.get_positions().tolist()
    }
geometry_record(atoms)

{'z': [8, 1, 1, 8, 1, 1, 8, 1, 1],
 'n_water': 3,
 'n_atom': 9,
 'atom': [0, 1, 1, 0, 1, 1, 0, 1, 1],
 'coords': [[25.3875809, 2.28446364, 8.01933861],
  [24.686451, 2.11461496, 7.36908007],
  [26.1070786, 1.70453322, 7.77935553],
  [22.9643402, 1.68695939, 6.75715494],
  [22.7494984, 1.67431045, 7.70416498],
  [22.2382431, 2.13693213, 6.33168697],
  [23.0780773, 1.86950338, 9.5477314],
  [22.9238548, 2.4637537, 10.2781725],
  [23.9850082, 2.04813766, 9.2500248]]}

Generate many records in parallel

In [16]:
rng_split = np.random.RandomState(4)
rng_pull = np.random.RandomState(129)

In [17]:
# Create the output files
filenames = [
    'geom_train', 'geom_test', 'geom_valid',
    'atomic_train', 'atomic_test', 'atomic_valid',
    'coarse_train', 'coarse_test', 'coarse_valid'
]
make_output = lambda x: tf.io.TFRecordWriter(os.path.join('data', 'output', f'{x}.proto'))
output_files = dict((x, make_output(x)) for x in filenames)
make_output = lambda x: gzip.open(os.path.join('data', 'output', f'{x}.json.gz'), 'wt')
json_outputs = dict((x, make_output(x)) for x in filenames)

# Control functions
batch_size = 4096
entry_gen = pull_from_database(ase_db, rng_pull)
counter = tqdm(total=len(ase_db))
coarse_fun = partial(make_nfp_network, coarsen=True)

try:
    done = False
    with Pool(n_jobs - 1) as p:  # One CPU open for serialization
        while not done:
            # Get the next batch of entries 
            batch = []
            for _ in range(batch_size):
                try:
                    batch.append(next(entry_gen))
                except StopIteration:
                    done = True
                    break
            
            # Make the random choices
            split_rnd = rng_split.random(len(batch))
            
            # Save the geometries
            name = 'geom'
            for atoms, r in zip(batch, split_rnd):
                # Make the record
                entry = geometry_record(atoms)
                serial_entry = make_tfrecord(entry)
                
                # Store in a specific dataset
                if r < val_fraction:
                    out_name = f'{name}_valid'
                elif r < val_fraction + test_fraction:
                    out_name = f'{name}_test'
                else:
                    out_name = f'{name}_train'
                    
                # Save to file
                output_files[out_name].write(serial_entry)
                print(json.dumps(entry), file=json_outputs[out_name])    
                
            
            # Process for both atomic and coarse_network
            for name, func in zip(['atomic', 'coarse'], [make_nfp_network, coarse_fun]):
                for atoms, entry, r in zip(batch, p.imap(func, batch, chunksize=64), split_rnd):
                    # Serialize the entry
                    serial_entry = make_tfrecord(entry)

                    # Store in a specific dataset
                    if r < val_fraction:
                        out_name = f'{name}_valid'
                    elif r < val_fraction + test_fraction:
                        out_name = f'{name}_test'
                    else:
                        out_name = f'{name}_train'
                    
                    # Save to file
                    output_files[out_name].write(serial_entry)
                    print(json.dumps(entry), file=json_outputs[out_name])
                        
            # Update TQDM
            counter.update(len(batch))
finally:
    for out in output_files.values():
        out.close()
    for out in json_outputs.values():
        out.close()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4464740/4464740 [8:49:53<00:00, 77.59it/s]